# 3D RD + Vase Boundary + Gradient + Tunnels

## Marching Cubes → Mesh → Interactive Preview

This notebook implements a **3D reaction-diffusion simulation** inside a vase-shaped boundary with:
- **Vertical nutrient gradient**: Feed (F) increases toward bottom, Kill (K) increases toward top
- **Agent-based tunnelling**: Random walk agents carve tunnels through the RD field
- **Marching cubes meshing**: Converts the 3D scalar field to a mesh
- **Interactive controls**: Jupyter widgets for real-time parameter exploration

### Features
- 3D Gray-Scott reaction-diffusion in a tapered vase boundary
- Spatial variation of F and K parameters based on height
- Multiple random seed patches for pattern generation
- Agent-based tunnel carving with configurable agents
- Automatic mesh scaling to target height
- Laplacian mesh smoothing for clean surfaces

## Imports

Install required packages if needed:
```bash
pip install numpy scipy scikit-image trimesh ipywidgets
```

In [2]:
# ============================================================
# Imports
# ============================================================
import numpy as np
from scipy.ndimage import convolve
from skimage.measure import marching_cubes
import trimesh
from trimesh.smoothing import filter_laplacian
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox
from IPython.display import display
import os
import uuid
from pathlib import Path

print("✓ Imports complete")

✓ Imports complete


## 1. Mesh Viewer (trimesh-based)

Replaces Colab's Three.js viewer with trimesh's scene viewer that works in Jupyter notebooks.

In [3]:
def show_mesh_trimesh(mesh, width=700, height=700):
    """
    Display a mesh using trimesh's scene viewer in Jupyter.
    
    Args:
        mesh: trimesh.Trimesh object
        width: Display width in pixels
        height: Display height in pixels
    """
    scene = mesh.scene()
    
    # Configure scene for better viewing
    scene.camera.resolution = (width, height)
    scene.camera.fov = (45, 45)  # Field of view
    
    # Show the scene (opens in Jupyter)
    return scene.show()

## 2. Vase Mask + Nutrient Gradient + RD Core

In [4]:
def make_vase_mask(n=64, radius_frac=0.7, taper=0.3):
    """
    Create a vase-like mask in an n^3 grid.
    
    Args:
        n: Grid resolution (n×n×n)
        radius_frac: Base radius as fraction of half-grid (0-1)
        taper: Taper factor (0 = cylinder, higher = stronger taper toward top)
    
    Returns:
        Tuple of (mask, Z_norm) where:
        - mask: Boolean 3D array (True inside vase)
        - Z_norm: Normalised Z coordinates (0 bottom, 1 top)
    """
    lin = np.linspace(-1, 1, n)
    X, Y, Z = np.meshgrid(lin, lin, lin, indexing="ij")
    
    R_xy = np.sqrt(X**2 + Y**2)
    Z_norm = (Z + 1.0) / 2.0  # 0 bottom, 1 top
    
    base_radius = radius_frac
    radius_z = base_radius * (1.0 - taper * Z_norm)
    
    mask = R_xy <= radius_z
    return mask.astype(bool), Z_norm

print("✓ Vase mask function defined")

✓ Vase mask function defined


In [5]:
# 6-neighbour Laplacian kernel for 3D diffusion
laplacian_kernel = np.zeros((3, 3, 3), dtype=np.float32)
laplacian_kernel[1,1,1] = -6
laplacian_kernel[0,1,1] = laplacian_kernel[2,1,1] = 1
laplacian_kernel[1,0,1] = laplacian_kernel[1,2,1] = 1
laplacian_kernel[1,1,0] = laplacian_kernel[1,1,2] = 1

print("✓ Laplacian kernel defined")

✓ Laplacian kernel defined


In [6]:
def initialise_fields(n, num_seeds, seed_radius, noise_level, mask):
    """
    Initialise U,V within the mask with multiple random spherical seeds.
    
    Args:
        n: Grid resolution
        num_seeds: Number of seed patches
        seed_radius: Radius of each seed patch (in grid units)
        noise_level: Noise amplitude (0-1)
        mask: Boolean mask defining valid region
    
    Returns:
        Tuple of (U, V) fields
    """
    U = np.ones((n, n, n), dtype=np.float32)
    V = np.zeros((n, n, n), dtype=np.float32)
    
    rng = np.random.default_rng()
    
    # Precompute coordinate grid for seed distances
    X, Y, Z = np.meshgrid(np.arange(n), np.arange(n), np.arange(n), indexing="ij")
    
    # Get all valid coordinates within mask
    xs, ys, zs = np.where(mask)
    coords = np.vstack((xs, ys, zs)).T
    
    # Place seed patches
    for _ in range(num_seeds):
        cx, cy, cz = coords[rng.integers(0, len(coords))]
        
        R = np.sqrt((X - cx)**2 + (Y - cy)**2 + (Z - cz)**2)
        seed = (R < seed_radius) & mask
        
        V[seed] = 1.0
        U[seed] = 0.5
    
    # Add noise
    if noise_level > 0:
        V += noise_level * rng.standard_normal(V.shape).astype(np.float32)
        np.clip(V, 0.0, 1.0, out=V)
    
    # Ensure mask boundary conditions
    V[~mask] = 0.0
    U[~mask] = 1.0
    
    return U, V

print("✓ Field initialisation function defined")

✓ Field initialisation function defined


In [7]:
def make_FK_fields(F, K, Z_norm, grad):
    """
    Create spatial F and K fields with a vertical gradient.
    
    Args:
        F: Base feed rate
        K: Base kill rate
        Z_norm: Normalised Z coordinates (0 bottom, 1 top)
        grad: Gradient strength (0 = uniform, higher = more variation)
    
    Returns:
        Tuple of (F_field, K_field) as 3D arrays
    """
    # More feed at bottom, more kill at top
    F_field = F * (1.0 + grad * (1.0 - Z_norm))
    K_field = K * (1.0 + grad * Z_norm)
    return F_field.astype(np.float32), K_field.astype(np.float32)

print("✓ Gradient field function defined")

✓ Gradient field function defined


In [8]:
def rd_step(U, V, Du, Dv, F_field, K_field, mask):
    """
    One Gray–Scott RD step with spatial F,K and vase mask.
    
    Args:
        U: U field (3D array)
        V: V field (3D array)
        Du: Diffusion rate for U
        Dv: Diffusion rate for V
        F_field: Spatial feed rate field (3D array)
        K_field: Spatial kill rate field (3D array)
        mask: Boolean mask defining valid region
    
    Returns:
        Tuple of (U_new, V_new) updated fields
    """
    # Compute Laplacians
    Lu = convolve(U, laplacian_kernel, mode="nearest")
    Lv = convolve(V, laplacian_kernel, mode="nearest")
    UVV = U * V * V
    
    # Gray-Scott update with spatial F and K
    U_new = U + (Du * Lu - UVV + F_field * (1.0 - U))
    V_new = V + (Dv * Lv + UVV - (F_field + K_field) * V)
    
    # Clamp to valid range
    np.clip(U_new, 0.0, 1.0, out=U_new)
    np.clip(V_new, 0.0, 1.0, out=V_new)
    
    # Apply mask boundary conditions
    V_new[~mask] = 0.0
    U_new[~mask] = 1.0
    
    return U_new, V_new

print("✓ RD step function defined")

✓ RD step function defined


In [9]:
def run_simulation(
    n=64,
    steps=120,
    F=0.035,
    K=0.065,
    num_seeds=5,
    seed_radius=6,
    noise_level=0.05,
    radius_frac=0.7,
    taper=0.3,
    grad=0.5,
):
    """
    Run RD inside a vase mask with vertical nutrient gradient.
    
    Args:
        n: Grid resolution
        steps: Number of simulation steps
        F: Base feed rate
        K: Base kill rate
        num_seeds: Number of seed patches
        seed_radius: Radius of seed patches
        noise_level: Initial noise amplitude
        radius_frac: Vase radius fraction
        taper: Vase taper factor
        grad: Nutrient gradient strength
    
    Returns:
        Tuple of (field, mask) where field is normalised V field
    """
    # Create vase mask
    mask, Z_norm = make_vase_mask(n=n, radius_frac=radius_frac, taper=taper)
    
    # Create spatial F and K fields
    F_field, K_field = make_FK_fields(F, K, Z_norm, grad)
    
    # Initialise fields
    U, V = initialise_fields(
        n=n,
        num_seeds=num_seeds,
        seed_radius=seed_radius,
        noise_level=noise_level,
        mask=mask,
    )
    
    # Diffusion rates
    Du, Dv = 0.16, 0.08
    
    # Run simulation
    for _ in range(steps):
        U, V = rd_step(U, V, Du, Dv, F_field, K_field, mask)
    
    # Normalise field for visualisation
    field = (V - V.min()) / (V.max() - V.min() + 1e-9)
    field[~mask] = 0.0
    
    return field, mask

print("✓ Simulation function defined")

✓ Simulation function defined


## 3. Agent-Based Tunnelling

In [10]:
def carve_tunnels(field, mask, n_agents, agent_steps, radius):
    """
    Simple random-walk tunnel carving: agents reduce field values inside a small radius.
    
    Args:
        field: 3D scalar field
        mask: Boolean mask defining valid region
        n_agents: Number of tunnelling agents
        agent_steps: Number of steps each agent takes
        radius: Tunnel radius (in grid units)
    
    Returns:
        Modified field with tunnels carved
    """
    if n_agents <= 0 or agent_steps <= 0:
        return field
    
    n = field.shape[0]
    rng = np.random.default_rng()
    coords = np.array(np.where(mask)).T
    
    f = field.copy()
    
    for _ in range(n_agents):
        # Start agent at random position within mask
        x, y, z = coords[rng.integers(0, len(coords))]
        
        for _ in range(agent_steps):
            # Define carving region
            xs = slice(max(0, x - radius), min(n, x + radius + 1))
            ys = slice(max(0, y - radius), min(n, y + radius + 1))
            zs = slice(max(0, z - radius), min(n, z + radius + 1))
            
            # Create distance mask for spherical carving
            X, Y, Z = np.meshgrid(
                np.arange(xs.start, xs.stop),
                np.arange(ys.start, ys.stop),
                np.arange(zs.start, zs.stop),
                indexing="ij",
            )
            R = np.sqrt((X - x)**2 + (Y - y)**2 + (Z - z)**2)
            carve_mask = R <= radius
            
            # Reduce field values in tunnel region (70% reduction)
            f[xs, ys, zs][carve_mask] *= 0.3
            
            # Random walk step
            dx, dy, dz = rng.integers(-1, 2, size=3)
            x = int(np.clip(x + dx, 0, n - 1))
            y = int(np.clip(y + dy, 0, n - 1))
            z = int(np.clip(z + dz, 0, n - 1))
            
            # If agent leaves mask, restart at random position
            if not mask[x, y, z]:
                x, y, z = coords[rng.integers(0, len(coords))]
    
    return f

print("✓ Tunnelling function defined")

✓ Tunnelling function defined


## 4. Field → Mesh Conversion

In [11]:
def field_to_mesh(field, iso, height_mm):
    """
    Convert 3D scalar field to mesh using marching cubes.
    
    Args:
        field: 3D scalar field
        iso: Isosurface threshold (0-1)
        height_mm: Target height in millimetres
    
    Returns:
        trimesh.Trimesh object, scaled to target height
    """
    # Extract isosurface
    verts, faces, normals, values = marching_cubes(field, level=iso)
    
    # Create mesh
    mesh = trimesh.Trimesh(vertices=verts, faces=faces, process=True)
    
    # Scale to target height
    max_extent = mesh.extents.max()
    if max_extent > 0:
        mesh.apply_scale(height_mm / max_extent)
    
    # Apply Laplacian smoothing
    filter_laplacian(mesh, lamb=0.5, iterations=1)
    
    return mesh

print("✓ Mesh conversion function defined")

✓ Mesh conversion function defined


## 5. Main Simulation Pipeline

In [ ]:
def simulate(
    F=0.035,
    K=0.065,
    steps=120,
    iso=0.5,
    height_mm=150,
    num_seeds=5,
    seed_radius=6,
    noise_level=0.05,
    radius_frac=0.7,
    taper=0.3,
    grad=0.5,
    n_agents=10,
    agent_steps=100,
    tunnel_radius=1,
    export=False,
    output_dir="outputs/meshes/stl",
):
    """
    Top-level driver: RD → tunnels → mesh → preview (+ optional STL export).
    
    Args:
        F: Base feed rate
        K: Base kill rate
        steps: Number of RD simulation steps
        iso: Isosurface threshold (0-1)
        height_mm: Target mesh height in millimetres
        num_seeds: Number of seed patches
        seed_radius: Radius of seed patches
        noise_level: Initial noise amplitude
        radius_frac: Vase radius fraction
        taper: Vase taper factor
        grad: Nutrient gradient strength
        n_agents: Number of tunnelling agents
        agent_steps: Steps per agent
        tunnel_radius: Radius of tunnels (grid units)
        export: Whether to export STL file
        output_dir: Directory for exported STL files
    
    Returns:
        Generated mesh
    """
    import time
    start_time = time.time()
    
    print("Running RD simulation...")
    print(f"  Grid: 64³, Steps: {steps}")
    field, mask = run_simulation(
        n=64,
        steps=steps,
        F=F,
        K=K,
        num_seeds=num_seeds,
        seed_radius=seed_radius,
        noise_level=noise_level,
        radius_frac=radius_frac,
        taper=taper,
        grad=grad,
    )
    rd_time = time.time() - start_time
    print(f"  ✓ RD complete ({rd_time:.1f}s)")
    
    print("Carving tunnels...")
    print(f"  Agents: {n_agents}, Steps/agent: {agent_steps}")
    tunnel_start = time.time()
    field = carve_tunnels(field, mask, n_agents, agent_steps, tunnel_radius)
    tunnel_time = time.time() - tunnel_start
    print(f"  ✓ Tunnels complete ({tunnel_time:.1f}s)")
    
    print("Meshing...")
    mesh_start = time.time()
    mesh = field_to_mesh(field, iso, height_mm)
    mesh_time = time.time() - mesh_start
    print(f"  ✓ Mesh complete ({mesh_time:.1f}s)")
    
    # Save preview
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    preview_path = output_path / "preview.stl"
    mesh.export(str(preview_path))
    print(f"Preview saved: {preview_path}")
    
    # Display mesh in Jupyter
    print("Rendering preview...")
    show_mesh_trimesh(mesh)
    
    # Optional export
    if export:
        name = f"rd_vase_F{F:.3f}_K{K:.3f}_s{steps}.stl"
        export_path = output_path / name
        mesh.export(str(export_path))
        print(f"Exported STL: {export_path}")
    
    total_time = time.time() - start_time
    print(f"\n⏱️  Total time: {total_time:.1f}s")
    print(f"   Mesh: {len(mesh.vertices):,} vertices, {len(mesh.faces):,} faces")
    
    return mesh

print("✓ Simulation pipeline defined")

✓ Simulation pipeline defined


## 6. Interactive Controls

Use the sliders below to explore different parameter combinations. The simulation will run automatically when you change any parameter.

**Performance Tips:**
- **Default values are optimised for speed** (60 RD steps, 5 agents, 50 steps/agent)
- For **faster exploration**: Keep steps ≤ 60, agents ≤ 5
- For **higher quality**: Increase steps to 120-200, agents to 10-20
- **Typical runtime**: 5-15 seconds with defaults, 20-60 seconds with high quality settings
- Progress indicators show timing for each stage

In [ ]:
# Create output directory
output_dir = Path("outputs/meshes/stl")
output_dir.mkdir(parents=True, exist_ok=True)

# Wrapper function that captures output_dir from closure
def simulate_with_fixed_output_dir(
    F=0.035,
    K=0.065,
    steps=60,  # Reduced default for faster exploration
    iso=0.5,
    height_mm=150,
    num_seeds=5,
    seed_radius=6,
    noise_level=0.05,
    radius_frac=0.7,
    taper=0.3,
    grad=0.5,
    n_agents=5,  # Reduced default for faster exploration
    agent_steps=50,  # Reduced default for faster exploration
    tunnel_radius=1,
    export=False,
):
    """Wrapper that fixes output_dir from closure."""
    return simulate(
        F=F,
        K=K,
        steps=steps,
        iso=iso,
        height_mm=height_mm,
        num_seeds=num_seeds,
        seed_radius=seed_radius,
        noise_level=noise_level,
        radius_frac=radius_frac,
        taper=taper,
        grad=grad,
        n_agents=n_agents,
        agent_steps=agent_steps,
        tunnel_radius=tunnel_radius,
        export=export,
        output_dir=output_dir,  # Use closure variable
    )

# Interactive widget interface
# Note: Lower defaults for faster interactive exploration
# Increase steps/agents for higher quality results
interact(
    simulate_with_fixed_output_dir,
    F=FloatSlider(min=0.020, max=0.080, step=0.001, value=0.035, description="F base"),
    K=FloatSlider(min=0.045, max=0.080, step=0.001, value=0.065, description="k base"),
    steps=IntSlider(min=40, max=200, step=10, value=60, description="RD steps"),  # Reduced from 120
    iso=FloatSlider(min=0.3, max=0.7, step=0.01, value=0.5, description="Iso level"),
    height_mm=FloatSlider(min=80, max=250, step=10, value=150, description="Height (mm)"),
    num_seeds=IntSlider(min=1, max=20, step=1, value=5, description="# seeds"),
    seed_radius=IntSlider(min=3, max=12, step=1, value=6, description="Seed radius"),
    noise_level=FloatSlider(min=0.00, max=0.20, step=0.01, value=0.05, description="Noise"),
    radius_frac=FloatSlider(min=0.3, max=0.9, step=0.05, value=0.7, description="Vase radius"),
    taper=FloatSlider(min=0.0, max=0.8, step=0.05, value=0.3, description="Vase taper"),
    grad=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5, description="Nutrient gradient"),
    n_agents=IntSlider(min=0, max=40, step=1, value=5, description="# tunnel agents"),  # Reduced from 10
    agent_steps=IntSlider(min=0, max=200, step=10, value=50, description="Steps/agent"),  # Reduced from 100
    tunnel_radius=IntSlider(min=1, max=3, step=1, value=1, description="Tunnel radius"),
    export=Checkbox(value=False, description="Export STL"),
)

## Quick Test Run

Run a quick simulation with default parameters:

In [ ]:
# Quick test with default parameters
mesh = simulate(
    F=0.035,
    K=0.065,
    steps=120,
    iso=0.5,
    height_mm=150,
    num_seeds=5,
    seed_radius=6,
    noise_level=0.05,
    radius_frac=0.7,
    taper=0.3,
    grad=0.5,
    n_agents=10,
    agent_steps=100,
    tunnel_radius=1,
    export=True,
)

print(f"\nMesh statistics:")
print(f"  Vertices: {len(mesh.vertices):,}")
print(f"  Faces: {len(mesh.faces):,}")
print(f"  Watertight: {mesh.is_watertight}")
print(f"  Volume: {mesh.volume:.2f} mm³")
print(f"  Surface area: {mesh.area:.2f} mm²")